<a href="https://colab.research.google.com/github/Arturo-Granados/Machine-Learning/blob/main/Homework_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 8: Deep learning 

In this homework, we'll build a model for predicting if we have an image of a dino or a dragon. For this, we will use the "Dino or Dragon?" dataset that can be downloaded from [Kaggle](kaggle datasets download -d agrigorev/dino-or-dragon).


Set up

In [1]:
#Import the main libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#Get data

In [2]:
#Import kaggle
!pip install kaggle 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#Check my actual directory
!pwd 

/content


In [4]:
#Import os module
import os 

#Directory configuration
os.environ['KAGGLE_CONFIG_DIR'] = '/content' 

In [5]:
!kaggle datasets download -d agrigorev/dino-or-dragon

100% 104M/104M [00:07<00:00, 12.6MB/s]
100% 104M/104M [00:07<00:00, 13.7MB/s]


## Extrac data form the zip file

In [6]:
#Import ZipFile form zipfile
from zipfile import ZipFile 

#Function to extra files from a zip file
def open_zipfile(file_name): 
  with ZipFile(file_name, 'r') as zip:
   
    #zip.printdir() 
    zip.extractall() 

In [7]:
#Use the open_zipfile funtion to extract the csv files 
open_zipfile('dino-or-dragon.zip')

# Data Preparation

The dataset contains around 1900 images of dinos and around 1900 images of dragons.

The dataset contains separate folders for training and validation.

# Model
For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(150, 150, 3)`
* Next, create a convolutional layer ([`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling ([`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/))
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a `Dense` layer with 64 neurons and `'relu'` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use [`SGD`](https://keras.io/api/optimizers/sgd/) with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`

For clarification about kernel size and max pooling, check [Office Hours](https://www.youtube.com/watch?v=1WRgdBTUaAc).

# Question 1

Since we have a binary classification problem, what is the best loss function for us?

- `binary crossentropy`
- `focal loss`
- `mean squared error`
- `categorical crossentropy`

Note: since we specify an activation for the output layer, we don't need to set `from_logits=True`

## Answer: binary crossentropy

# Question 2
What's the total number of parameters of the model? You can use the summary method for that.

* 9215873
* 11215873
* 14215873
* 19215873

In [8]:
#Import libraries for CNN model 
import tensorflow as tf 
from tensorflow import keras

In [36]:
model =  tf.keras.Sequential([
    keras.layers.Conv2D(filters = 32,kernel_size = (3, 3), activation = 'relu', input_shape=(150, 150, 3)),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(units = 64, activation = 'relu'),
    keras.layers.Dense(units = 1, activation = 'sigmoid')
])

In [37]:
model.compile(
    optimizer = tf.keras.optimizers.SGD(lr=0.002, momentum=0.8),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']

)

In [38]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 175232)            0         
                                                                 
 dense_2 (Dense)             (None, 64)                11214912  
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
__________________________________________

##Answer: 11,215,873

# Generators and Training

For the next two questions, use the following data generator for both train and validation:

```python
ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/val directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and validation 

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)
```

In [39]:
#Def image generator
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [40]:
import pathlib
#Def data paths
train_dir = '/content/train'  
test_dir = '/content/test'

#Define train, test, validation data paths dir 
train_dir = pathlib.Path(train_dir)
test_dir = pathlib.Path(test_dir)

In [41]:
#Def train set 
train_ds = img_gen.flow_from_directory(train_dir,
                                       target_size=(150, 150),
                                       batch_size=20,
                                       shuffle=True,
                                       class_mode='binary')
#Def tests set 
test_ds = img_gen.flow_from_directory(test_dir,
                                      target_size=(150, 150),
                                      batch_size=20,
                                      shuffle=True,
                                      class_mode='binary')

Found 1594 images belonging to 2 classes.
Found 394 images belonging to 2 classes.


In [42]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds
)

Epoch 1/10
80/80 [==============================] - 9s 113ms/step - loss: 0.5923 - accuracy: 0.6662 - val_loss: 0.4680 - val_accuracy: 0.8223
Epoch 2/10
80/80 [==============================] - 9s 111ms/step - loss: 0.4697 - accuracy: 0.7936 - val_loss: 0.4320 - val_accuracy: 0.8071
Epoch 3/10
80/80 [==============================] - 9s 111ms/step - loss: 0.3819 - accuracy: 0.8432 - val_loss: 0.3488 - val_accuracy: 0.8680
Epoch 4/10
80/80 [==============================] - 9s 111ms/step - loss: 0.3163 - accuracy: 0.8701 - val_loss: 0.3650 - val_accuracy: 0.8223
Epoch 5/10
80/80 [==============================] - 9s 111ms/step - loss: 0.3073 - accuracy: 0.8858 - val_loss: 0.3086 - val_accuracy: 0.8731
Epoch 6/10
80/80 [==============================] - 9s 111ms/step - loss: 0.2586 - accuracy: 0.9003 - val_loss: 0.2933 - val_accuracy: 0.8782
Epoch 7/10
80/80 [==============================] - 9s 112ms/step - loss: 0.2236 - accuracy: 0.9166 - val_loss: 0.3032 - val_accuracy: 0.8655
Epoch 

# Question 3

What is the median of training accuracy for all the epochs for this model?

- 0.40
- 0.60
- 0.90
- 0.20


In [43]:
np.mean(history.history['accuracy'])

0.8700752675533294

# Question 4

What is the standard deviation of training loss for all the epochs for this model?

- 0.11
- 0.66
- 0.99
- 0.33

In [44]:
np.std(history.history['loss'])

0.13307542942969783

# Data Augmentation

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=40,`
* `width_shift_range=0.2,`
* `height_shift_range=0.2,`
* `shear_range=0.2,`
* `zoom_range=0.2,`
* `horizontal_flip=True,`
* `fill_mode='nearest'

In [45]:
train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [46]:
test_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [47]:
#Def train set 
train_ds = train_data_generator.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    shuffle=True,
                                                    class_mode='binary')

Found 1594 images belonging to 2 classes.


In [48]:
#Def tests set 
test_ds = test_data_generator.flow_from_directory(test_dir,
                                                  target_size=(150, 150),
                                                  batch_size=20,
                                                  shuffle=True,
                                                  class_mode='binary')

Found 394 images belonging to 2 classes.


In [49]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds,
    validation_steps=50)

Epoch 1/10
80/80 [==============================] - ETA: 0s - loss: 4154.8345 - accuracy: 0.5458

80/80 [==============================] - 16s 201ms/step - loss: 4154.8345 - accuracy: 0.5458 - val_loss: 126.2163 - val_accuracy: 0.4975
Epoch 2/10
80/80 [==============================] - 14s 170ms/step - loss: 0.7798 - accuracy: 0.5138
Epoch 3/10
80/80 [==============================] - 14s 170ms/step - loss: 0.6928 - accuracy: 0.5188
Epoch 4/10
80/80 [==============================] - 14s 171ms/step - loss: 0.6930 - accuracy: 0.5188
Epoch 5/10
80/80 [==============================] - 14s 169ms/step - loss: 0.6928 - accuracy: 0.5188
Epoch 6/10
80/80 [==============================] - 14s 170ms/step - loss: 0.6928 - accuracy: 0.5188
Epoch 7/10
80/80 [==============================] - 14s 173ms/step - loss: 0.6931 - accuracy: 0.5188
Epoch 8/10
80/80 [==============================] - 14s 170ms/step - loss: 0.6926 - accuracy: 0.5188
Epoch 9/10
80/80 [==============================] - 14s 171ms/step - loss: 0.6926 - accuracy: 0.5188
Epoch 10/10
80/80 [==============================] - 13

# Question 5 

Let's train our model for 10 more epochs using the same code as previously.
Make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of validation loss for all the epochs for the model trained with augmentations?

- 0.15
- 0.77
- 0.37
- 0.97

In [55]:
np.mean(history.history['loss'])

0.72


# Question 6

What's the average of validation accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

- 0.84
- 0.54
- 0.44
- 0.24

In [54]:
np.mean(history.history['accuracy'][5:])

0.518820583820343